<a href="https://colab.research.google.com/github/tomeliud/test_project/blob/master/Python_Notebook_Moringa_Data_Science_Prep_W4_Independent_Project_2020_05_TOM_NJOROGE_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Moringa Data Science Prep Independent Project: Week 4


Research Question

Identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.

Bonus Questions (Optional)

What is the most popular hour for returning cars?
What station is the most popular?
Overall?
At the most popular picking hour?
What postal code is the most popular for picking up Blue cars? Does the most popular station belong to that postal code?
Overall?
At the most popular picking hour?
Do the results change if you consider Utilib and Utilib 1.4 instead of Blue cars? 

Dataset provided 
Autolib dataset analysis [http://bit.ly/autolib_dataset]

Importing our libraries 

In [0]:
import pandas as pd 
import numpy as np 

Loading our dataset

In [51]:
#Loading Autolib dataset 
autolib_dataset = pd.read_csv('http://bit.ly/autolib_dataset%5D', encoding= 'utf-8')
autolib_dataset.tail(5)

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
4995,8 avenue MÃ©nelotte,2,2,0,0,0,nonexistent,Colombes,NaN,colombes-menelotte-8,STATION,"48.9246525, 2.259313",92700,Colombes/MÃ©nelotte/8,operational,NaN,3,station,ok,nonexistent,2018,4,6,11,26
4996,37 rue de Dantzig,4,4,0,0,1,operational,Paris,NaN,paris-dantzig-37,STATION,"48.8335103, 2.2987201",75015,Paris/Dantzig/37,operational,NaN,2,station,ok,nonexistent,2018,4,4,16,56
4997,142 rue du Bac,1,1,0,0,1,operational,Paris,NaN,paris-bac-142,STATION,"48.8508194, 2.3237968",75007,Paris/Bac/142,operational,NaN,4,station,ok,nonexistent,2018,4,1,7,1
4998,2 avenue du Val de Fontenay,2,2,0,0,0,nonexistent,Fontenay-Sous-Bois,NaN,fontenaysousbois-valdefontenay-2,STATION,"48.8528247, 2.4869085",94120,Fontenay-Sous-Bois/Val de Fontenay/2,operational,NaN,3,station,ok,nonexistent,2018,4,4,17,27
4999,5 Rue Henry Brisson,4,4,0,0,0,operational,Paris,NaN,paris-henrybrisson-5,STATION,"48.8984293, 2.3347923",75018,Paris/Henry Brisson/5,operational,NaN,0,station,ok,nonexistent,2018,4,8,10,2


Exploration 

In [52]:
#checking information in my dataset 
autolib_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

In [53]:
#finding description to our dataset eg. mean, sum, min, max
autolib_dataset.describe()

,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Postal code,Slots,year,month,day,hour,minute
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.0,5000.0,5000.000000,5000.000000,5000.000000
mean,2.333600,2.333600,0.059600,0.123000,0.253400,82634.878400,1.932400,2018.0,4.0,4.941600,11.509200,29.270000
std,2.035274,2.035274,0.246698,0.356506,0.546304,8835.865721,1.905402,0.0,0.0,2.597063,6.893549,17.231741
min,0.000000,0.000000,0.000000,0.000000,0.000000,75001.000000,0.000000,2018.0,4.0,1.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,75012.000000,0.000000,2018.0,4.0,3.000000,6.000000,14.000000
50%,2.000000,2.000000,0.000000,0.000000,0.000000,75019.000000,1.000000,2018.0,4.0,5.000000,11.000000,29.000000
75%,4.000000,4.000000,0.000000,0.000000,0.000000,92320.000000,3.000000,2018.0,4.0,7.000000,18.000000,44.000000
max,7.000000,7.000000,2.000000,3.000000,2.000000,95880.000000,7.000000,2018.0,4.0,9.000000,23.000000,59.000000


Data cleaning 

In [54]:
#droping columns that will not be needed in our anlysis 
car_service = autolib_dataset.drop(['Charge Slots','Charging Status','Rental status','Slots','year','month','minute','Cars','Displayed comment','Geo point','Scheduled at','ID'],axis=1)
car_service.head(5)

,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,Kind,Postal code,Public name,Station type,Status,Subscription status,day,hour
0,2 Avenue de Suffren,0,0,0,Paris,STATION,75015,Paris/Suffren/2,station,ok,nonexistent,8,11
1,145 Rue Raymond Losserand,6,0,0,Paris,STATION,75014,Paris/Raymond Losserand/145,station,ok,nonexistent,6,7
2,2 Avenue John Fitzgerald Kennedy,3,0,2,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,station,ok,nonexistent,3,20
3,51 Rue EugÃ¨ne OudinÃ©,3,1,0,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,station,ok,nonexistent,4,4
4,6 avenue de la Porte de Champerret,3,0,0,Paris,PARKING,75017,Paris/Porte de Champerret/6,station,ok,nonexistent,8,17


In [55]:
#finding duplicate rows 
car_service[car_service.duplicated()].tail(5)

,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,Kind,Postal code,Public name,Station type,Status,Subscription status,day,hour
4618,24 rue du ChÃ¢teau Landon,5,0,0,Paris,STATION,75010,Paris/ChÃ¢teau Landon/24,station,ok,nonexistent,5,5
4655,1 place Stalingrad,3,0,0,Neuilly-Plaisance,STATION,93360,Neuilly-Plaisance/Stalingrad/1,station,ok,nonexistent,2,2
4682,13 Rue de Lens,4,0,0,Nanterre,STATION,92000,Nanterre/Lens/13,station,ok,nonexistent,9,2
4756,80 rue de SÃ¨vres,3,0,0,Boulogne-Billancourt,STATION,92100,Boulogne-Billancourt/SÃ¨vres/80,station,ok,nonexistent,9,2
4813,29 Avenue Georges Pompidou,2,0,0,Villeneuve-la-Garenne,STATION,92390,Villeneuve-la-Garenne/Pompidou/29,station,ok,nonexistent,9,16


In [56]:
#droping duplicate columns 
car_service = car_service.drop_duplicates()
car_service.head(5)

,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,Kind,Postal code,Public name,Station type,Status,Subscription status,day,hour
0,2 Avenue de Suffren,0,0,0,Paris,STATION,75015,Paris/Suffren/2,station,ok,nonexistent,8,11
1,145 Rue Raymond Losserand,6,0,0,Paris,STATION,75014,Paris/Raymond Losserand/145,station,ok,nonexistent,6,7
2,2 Avenue John Fitzgerald Kennedy,3,0,2,Le Bourget,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,station,ok,nonexistent,3,20
3,51 Rue EugÃ¨ne OudinÃ©,3,1,0,Paris,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,station,ok,nonexistent,4,4
4,6 avenue de la Porte de Champerret,3,0,0,Paris,PARKING,75017,Paris/Porte de Champerret/6,station,ok,nonexistent,8,17


In [57]:
#checking for null values 
car_service.isnull().sum()

Address                0
Bluecar counter        0
Utilib counter         0
Utilib 1.4 counter     0
City                   0
Kind                   0
Postal code            0
Public name            0
Station type           0
Status                 0
Subscription status    0
day                    0
hour                   0
dtype: int64

In [58]:
#Changing the case of column names to lower, stripping whitespaces, removing periods and replacing spaces with underscore
car_service.columns = car_service.columns.str.lower().str.strip().str.replace(' ', '_').str.replace('.', '')
car_service.columns

Index(['address', 'bluecar_counter', 'utilib_counter', 'utilib_14_counter',
       'city', 'kind', 'postal_code', 'public_name', 'station_type', 'status',
       'subscription_status', 'day', 'hour'],
      dtype='object')

In [59]:
#Adding a use column for the three car counters (bluecar_counter, utilib_counter and utilib_14_counter)
#postive number means that a car was returned negative number means that a car was picked while 0 means that nothing happened (no car was picked nor returned)
#Creating column for bluecar usage
car_service['bluecar_use'] = car_service.bluecar_counter.diff()
#Creating column for utilib usage
car_service['utilib_use'] = car_service.utilib_counter.diff()
#Creating column for utilib 14 usage
car_service['utilib_14_use'] = car_service.utilib_14_counter.diff()
car_service.tail(5)

,address,bluecar_counter,utilib_counter,utilib_14_counter,city,kind,postal_code,public_name,station_type,status,subscription_status,day,hour,bluecar_use,utilib_use,utilib_14_use
4995,8 avenue MÃ©nelotte,2,0,0,Colombes,STATION,92700,Colombes/MÃ©nelotte/8,station,ok,nonexistent,6,11,2.0,0.0,0.0
4996,37 rue de Dantzig,4,0,0,Paris,STATION,75015,Paris/Dantzig/37,station,ok,nonexistent,4,16,2.0,0.0,0.0
4997,142 rue du Bac,1,0,0,Paris,STATION,75007,Paris/Bac/142,station,ok,nonexistent,1,7,-3.0,0.0,0.0
4998,2 avenue du Val de Fontenay,2,0,0,Fontenay-Sous-Bois,STATION,94120,Fontenay-Sous-Bois/Val de Fontenay/2,station,ok,nonexistent,4,17,1.0,0.0,0.0
4999,5 Rue Henry Brisson,4,0,0,Paris,STATION,75018,Paris/Henry Brisson/5,station,ok,nonexistent,8,10,2.0,0.0,0.0


Exploration of our new dataset

In [60]:
#checking information of the new dataset 
car_service.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4968 entries, 0 to 4999
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   address              4968 non-null   object 
 1   bluecar_counter      4968 non-null   int64  
 2   utilib_counter       4968 non-null   int64  
 3   utilib_14_counter    4968 non-null   int64  
 4   city                 4968 non-null   object 
 5   kind                 4968 non-null   object 
 6   postal_code          4968 non-null   int64  
 7   public_name          4968 non-null   object 
 8   station_type         4968 non-null   object 
 9   status               4968 non-null   object 
 10  subscription_status  4968 non-null   object 
 11  day                  4968 non-null   int64  
 12  hour                 4968 non-null   int64  
 13  bluecar_use          4967 non-null   float64
 14  utilib_use           4967 non-null   float64
 15  utilib_14_use        4967 non-null   f

In [61]:
#description of our new dataset 
car_service.describe()

,bluecar_counter,utilib_counter,utilib_14_counter,postal_code,day,hour,bluecar_use,utilib_use,utilib_14_use
count,4968.000000,4968.000000,4968.000000,4968.000000,4968.000000,4968.000000,4967.000000,4967.000000,4967.000000
mean,2.335950,0.059581,0.122987,82645.133857,4.938205,11.520733,0.000805,0.000000,0.000000
std,2.035833,0.246728,0.356666,8837.985267,2.596401,6.888275,2.855071,0.350479,0.502511
min,0.000000,0.000000,0.000000,75001.000000,1.000000,0.000000,-7.000000,-2.000000,-3.000000
25%,0.000000,0.000000,0.000000,75012.000000,3.000000,6.000000,-2.000000,0.000000,0.000000
50%,2.000000,0.000000,0.000000,75019.000000,5.000000,11.000000,0.000000,0.000000,0.000000
75%,4.000000,0.000000,0.000000,92320.000000,7.000000,18.000000,2.000000,0.000000,0.000000
max,7.000000,2.000000,3.000000,95880.000000,9.000000,23.000000,7.000000,2.000000,3.000000


In [62]:
#finding the shape of our new dataset 
car_service.shape

(4968, 16)

Analysis questions 

Bluecars Analysis

Question 1 


What is the most popular hour of the day for picking up a shared electric car in the city of Paris over the month of April 2018 ? (Specifically Bluecars)

In [63]:
#question 1 above
car_service.bluecar_use.unique()
#The most popular hour of the day for picking cars is when the difference is at maximum (-7) is 1300 hours
car_service[(car_service.city == 'Paris') & (car_service.bluecar_use == -7.0)].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

hour
14    1
Name: hour, dtype: int64

Question 2

What is the most popular hour for returning cars?

In [64]:
car_service[car_service.bluecar_use == 7.].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

hour
0    4
Name: hour, dtype: int64

Question 2 

What station is the most popular overall ?

In [65]:
car_service[(car_service.kind == 'STATION') & (car_service.status == 'ok')].groupby('public_name')[['kind']].count().sort_values(by = 'kind', ascending  = False).head(1)

,kind
public_name,
Paris/Porte de Montrouge/8,13


Question 3 

What is the most popular station at the most popular hour ?



In [66]:
car_service[(car_service.kind == 'STATION') & (car_service.hour == 13)  & (car_service.status == 'ok')].groupby('public_name').count()[['kind']].sort_values(by = 'kind', ascending = False).head(1)

,kind
public_name,
Issy-les-Moulineaux/Vaudetard/7,2


Question 4 

What postal code is most popular when picking up Bluecars?

In [67]:
car_service[(car_service.bluecar_use < 0) & (car_service.hour == 13)].groupby('postal_code').count()[['bluecar_use']].sort_values(by = 'bluecar_use', ascending = False).head(1)

,bluecar_use
postal_code,
75008,8


Question 5 

Utilib Analysis
Using Utilib instead of Bluecars

What is the most popular hour of the day for picking up a shared electric car in the city of Paris over the month of April 2018 ? (Specifically Utilib)

In [68]:
car_service.utilib_use.unique()
#This is the range of the differences logged in the Utilib_use column. 
#The maximum negative number (-4) shows when the picking of the cars was highest.
#Similly, the maximum positive number (4) shows when the returning of the cars was highest.
car_service[(car_service.city == 'Paris') & (car_service.utilib_use.min() < 0 )].groupby('hour')['hour'].count().sort_values(ascending = False).head(1)

hour
21    145
Name: hour, dtype: int64

What postal code is most popular when picking up Utilib cars ?



In [69]:
car_service[car_service.utilib_use < 0].groupby('postal_code').count()[['utilib_use']].sort_values(by = 'utilib_use', ascending = False).head(1)

,utilib_use
postal_code,
75015,21


What postal code is the most popular for picking up Utilib cars at the most popular picking hour?

In [70]:
car_service[(car_service.utilib_use < 0) & (car_service.hour == 19)].groupby('postal_code').count()[['utilib_use']].sort_values(by = 'utilib_use', ascending = False).head(1)

,utilib_use
postal_code,
75020,4
